In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import ast

In [3]:
result = pd.read_csv('/home/anonymous/belka/domain_103_e5.csv')


result['binds'] = result['binds'].apply(ast.literal_eval)
result = result.set_index('id')
result[['BRD4', 'HSA', 'sEH']] = pd.DataFrame(result['binds'].tolist(), index=result.index)

# Drop the original list column if not needed
result_r = result.drop(columns=['binds'])


In [5]:
result = pd.read_csv('/home/anonymous/belka/r_226-e5.csv')

result['binds'] = result['binds'].apply(ast.literal_eval)
result = result.set_index('id')
result[['BRD4', 'HSA', 'sEH']] = pd.DataFrame(result['binds'].tolist(), index=result.index)

# Drop the original list column if not needed
result_u = result.drop(columns=['binds'])


In [4]:
TEST_MAIN = '/home/anonymous/belka/test_with_islands.parquet'

In [5]:
parquet_file = pq.ParquetFile(TEST_MAIN)

In [19]:
def clamp(n, min, max): 
    if n < min: 
        return min
    elif n > max: 
        return max
    else: 
        return n 

In [6]:
parquet_file = pq.ParquetFile(TEST_MAIN)
results = []
eps=1e-10
from tqdm.auto import tqdm
for batch in tqdm(parquet_file.iter_batches(batch_size=4096)):
    batch = batch.to_pandas()
     # Merge the batch DataFrame with the result DataFrame on 'id_numberic' and 'id'
    merged_df = batch.merge(result_r, left_on='molecule_smiles', right_index=True, how='left')
    # merged_df = merged_df.merge(result_u, left_on='molecule_smiles', right_index=True, how='left', suffixes=('_r', '_u'))

    def get_score(row):
        # return row[row['protein_name']]
        # return row[row['protein_name']+'_r']
        # # return row[row['protein_name']+'_u']*0.5+row[row['protein_name']+'_r']*0.5
        if row['island']!=1: 
        #     # return (row[row['protein_name']+'_u']+row[row['protein_name']+'_r'])/2
            return row[row['protein_name']]
        else:
        #     # return row[row['protein_name']+'_r']
            return 0 
        # else:
        #     return row[row['protein_name']+'_u']
    merged_df['binds'] = merged_df.apply(get_score, axis=1)
    # merged_df = merged_df.drop(['molecule_smiles','protein_name','BRD4_r','HSA_r','sEH_r','BRD4_u','HSA_u','sEH_u','buildingblock1_smiles','buildingblock2_smiles','buildingblock3_smiles'], axis=1)
    merged_df = merged_df.drop(['molecule_smiles','protein_name','BRD4','HSA','sEH','buildingblock1_smiles','buildingblock2_smiles','buildingblock3_smiles'], axis=1)
    results.append(merged_df)

/home/anonymous/miniconda3/envs/belka/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0it [00:00, ?it/s]

355it [01:19,  4.52it/s]

In [ ]:
final_result = pd.concat(results).drop(['island'],axis=1)
# final_result.to_csv('output_77_s1_mix_random_unique.csv', index=None)
final_result.to_csv('output_domain_103_e5.csv', index=None)


In [11]:
final_result
#model_emb_s1_random_0.71.csv
#model_emb_216_full.csv

,id,binds
0,295246830,8.692931e-05
1,295246831,2.885004e-03
2,295246832,4.377997e-04
3,295246833,2.899197e-05
4,295246834,1.732452e-03
...,...,...
3723,296921721,5.821172e-05
3724,296921722,1.666633e-04
3725,296921723,7.143399e-06
3726,296921724,7.363746e-05
